In [ ]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)
# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        "/content/drive/My Drive/dataset/kaggle_big_data_set/Fish_Dataset",
        target_size=(48, 48),
        batch_size=64,
        color_mode="rgb",
        class_mode='categorical')

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        "/content/drive/My Drive/dataset/kaggle_big_data_set/Fish_Dataset",
        target_size=(48, 48),
        batch_size=64,
        color_mode="rgb",
        class_mode='categorical')

#create model structure
fish_model = Sequential()

fish_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 3)))
fish_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
fish_model.add(MaxPooling2D(pool_size=(2, 2)))
fish_model.add(Dropout(0.25))

fish_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
fish_model.add(MaxPooling2D(pool_size=(2, 2)))
fish_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
fish_model.add(MaxPooling2D(pool_size=(2, 2)))
fish_model.add(Dropout(0.25))

fish_model.add(Flatten())
fish_model.add(Dense(1024, activation='relu'))
fish_model.add(Dropout(0.5))
fish_model.add(Dense(9, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

fish_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

#early_stop = EarlyStopping(monitor='val_loss',patience=1)

Found 18076 images belonging to 9 classes.
Found 18076 images belonging to 9 classes.


In [ ]:
fish_model_info = fish_model.fit(
        train_generator,
        steps_per_epoch=18076 // 64,
        epochs=30,
        validation_data=validation_generator,
        validation_steps=430 // 64,
        #callbacks=[early_stop]
        )


Epoch 1/30
282/282 [==============================] - 4256s 15s/step - loss: 1.4108 - accuracy: 0.4542 - val_loss: 0.9568 - val_accuracy: 0.6641
Epoch 2/30
282/282 [==============================] - 390s 1s/step - loss: 0.8892 - accuracy: 0.6632 - val_loss: 0.5491 - val_accuracy: 0.7995
Epoch 3/30
282/282 [==============================] - 371s 1s/step - loss: 0.7034 - accuracy: 0.7303 - val_loss: 0.6035 - val_accuracy: 0.7734
Epoch 4/30
282/282 [==============================] - 375s 1s/step - loss: 0.5784 - accuracy: 0.7860 - val_loss: 0.4120 - val_accuracy: 0.8385
Epoch 5/30
282/282 [==============================] - 368s 1s/step - loss: 0.4978 - accuracy: 0.8100 - val_loss: 0.2189 - val_accuracy: 0.9219
Epoch 6/30
282/282 [==============================] - 371s 1s/step - loss: 0.4367 - accuracy: 0.8354 - val_loss: 0.3021 - val_accuracy: 0.8828
Epoch 7/30
282/282 [==============================] - 368s 1s/step - loss: 0.3919 - accuracy: 0.8544 - val_loss: 0.2378 - val_accuracy: 0.92

In [ ]:
# save model structure in jason file
model_json = fish_model.to_json()
with open("fish_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
fish_model.save_weights('fish_model.h5')

# test verileri


In [ ]:
import os
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
test_data_dir = "/content/drive/My Drive/dataset/kaggle_big_data_set/NA_Fish_Dataset"

test_images = []
test_labels = []

for root, dirs, files in os.walk(test_data_dir):
    for filename in files:
        if filename.lower().endswith((".jpg", ".png")):
            img_path = os.path.join(root, filename)
            img = image.load_img(img_path, target_size=(48, 48), color_mode="rgb")
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = img_array / 255.0
            test_images.append(img_array)

            label = os.path.basename(root)
            test_labels.append(label)
if len(test_images) == 0:
    print("No images were processed.")
else:
    print(f"Processed {len(test_images)} images.")

Processed 439 images.


In [ ]:
test_images = np.vstack(test_images)
test_labels = np.array(test_labels)

print("Test veri seti boyutu:", test_images.shape)
print("Test etiketleri boyutu:", test_labels.shape)

label_encoder = LabelEncoder()
test_labels_encoded = label_encoder.fit_transform(test_labels)

num_classes = max(test_labels_encoded) + 1

test_labels_onehot = to_categorical(test_labels_encoded, num_classes=num_classes)
print(test_labels_onehot)

print(f"Number of classes: {num_classes}")
print(f"Classes: {label_encoder.classes_}")

Test veri seti boyutu: (439, 48, 48, 3)
Test etiketleri boyutu: (439,)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]]
Number of classes: 9
Classes: ['Black Sea Sprat' 'Gilt Head Bream' 'Horse Mackerel' 'Red Mullet'
 'Red Sea Bream' 'Sea Bass' 'Shrimp' 'Striped Red Mullet' 'Trout']


In [ ]:
json_file = open('/content/fish_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

In [ ]:
fish_model = model_from_json(loaded_model_json)
fish_model.load_weights('/content/fish_model.h5')
print("Model diskten yüklendi.")

Model diskten yüklendi.


In [ ]:
fish_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
loss, accuracy = fish_model.evaluate(test_images, test_labels_onehot, verbose=1)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy * 100:.2f}%")

14/14 [==============================] - 2s 93ms/step - loss: 1.5093 - accuracy: 0.7403
Test Loss: 1.5093
Test Accuracy: 74.03%
